In [108]:
import pandas as pd
import re

#### 상품별 리뷰 df

In [109]:
# 상품별 리뷰 정보 DF로 가져오기
goodsReviews_path = '/Users/ryeongjoo/Desktop/workspace/semi_prj/temp_result111.csv'
goods_df = pd.read_csv(goodsReviews_path, header=0, encoding='UTF-8')

/var/folders/mn/w592whfs2tb4g8s658288fb00000gn/T/ipykernel_36352/4067978522.py:3: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  goods_df = pd.read_csv(goodsReviews_path, header=0, encoding='UTF-8')


In [110]:
# 필요한 칼럼만 담은 DF
my_goods_df = goods_df[['gdasSeq', 'goodsNo', 'ordNo', 'ordGoodsSeq', 'gdasScrVal', 'gdasCont', 'mbrNo', 'photoGdasYn', 'gdasSctCd', 'goodsNm', 'dispRegDate', 'recommCnt', 'topRvrRnk', 'firstGdasYn', 'reviewerFmsRnkCat', 'evalList']]
# print(my_goods_df.head())

In [111]:
# NaN 값 드랍
my_goods_df = my_goods_df.dropna(subset=['gdasCont'])

# 영어 소문자 통일
my_goods_df['gdasCont'] = my_goods_df['gdasCont'].str.lower()

In [112]:
# 정규식을 이용하여 이모티콘 추출하는 함수
def extract_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001f600-\U0001f64f"  # 이모티콘
        u"\U0001f300-\U0001f5ff"  # 기호 및 문장 부호
        u"\U0001f680-\U0001f6ff"  # 기타 이모티콘
        u"\U0001f1e0-\U0001f1ff"  # 국기 및 기호
                           "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

# '이모티콘' 열 생성
my_goods_df['unicode'] = my_goods_df['gdasCont'].apply(lambda x: extract_emoji(x))

In [113]:
# 리뷰내용 중복문자, 이모티콘 제거
clean_reviews = []
emoticon_list = []

def remove_emoji(text):
    # 이모티콘 패턴 정의
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 이모티콘
                               u"\U0001F300-\U0001F5FF"  # 심볼
                               u"\U0001F680-\U0001F6FF"  # 트랜스포트 및 지도
                               u"\U0001F1E0-\U0001F1FF"  # 국기 및 이모지
                               "]+", flags=re.UNICODE)
    # 이모티콘 제거
    return emoji_pattern.sub(r'', text)

for review in my_goods_df['gdasCont'] :
    
    # 초성, 줄바꿈 제거
    review = re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', review)
    review = re.sub('[\r\n]', ' ', review)  
    
    # 이모티콘 제거
    review = remove_emoji(review)
    
    clean_reviews.append(review)
    
my_goods_df['gdasCont'] = clean_reviews

In [114]:
# 리뷰 길이 칼럼 추가
my_goods_df['cont_length'] = my_goods_df['gdasCont'].apply(lambda x: len(str(x)))

# print(my_goods_df.head())

In [115]:
# 음식 / 화장품으로 상품 분류한 칼럼 추가

import numpy as np

conditions = [
    (my_goods_df['reviewerFmsRnkCat'] == '건강식품, 푸드'),
    (my_goods_df['reviewerFmsRnkCat'] == '건강식품'),
    (my_goods_df['reviewerFmsRnkCat'] == '푸드'),
    (my_goods_df['reviewerFmsRnkCat'] == '헤어케어, 푸드'),
]
choices = ['food', 'food', 'food', 'food']
my_goods_df['reviewerFmsRnkCat_'] = np.select(conditions, choices, default='cosmetic')

my_goods_df['reviewerFmsRnkCat_'].fillna('cosmetic', inplace=True)

In [116]:
# evalList에서 'evalQstCont', 'evalAnsCont' 가져오기

# json은 str 형태로 넘어옴 -> 딕셔너리로 변환
import json

for eval in my_goods_df['evalList'].head(1):
    
    # 작은 따옴표를 큰 따옴표로, None을 null로 바꾸기
    fixed_str_data = eval.replace("'", "\"").replace("None", "null")

    # JSON 디코딩 -> list 형태로 바뀌며 인덱스마다 딕셔너리 하나씩
    eval_jason = json.loads(fixed_str_data)
    
    for eval_dict in eval_jason :
        print(eval_dict['evalQstCont'], eval_dict['evalAnsCd'], eval_dict['evalAnsCont'])

    

포장상태 A000000001325 꼼꼼해요
유통기한 A000000001328 예상보다 길어요
맛 A000000001331 아주 맛있어요


In [117]:
# for eval in my_goods_df['evalList'].head(1):
#     print(eval)

#### 상품별 df

In [118]:
product_df = pd.DataFrame()

In [119]:
# goodsNo 칼럼 생성
unique_goodsNo = my_goods_df['goodsNo'].unique()  # 중복을 제거한 goodsNo 값 추출
product_df['goodsNo'] = pd.Series(unique_goodsNo)

In [120]:
# 상품별 두달간 리뷰 평점 평균
scrMean_df = my_goods_df.groupby('goodsNo')['gdasScrVal'].mean().reset_index().rename(columns={'gdasScrVal': 'scrMean'})

# product_df와 scrMean_df 조인
product_df = pd.merge(product_df, scrMean_df, on='goodsNo', how='left')

In [122]:
product_df

,goodsNo,scrMean
0,A000000177512,9.808000
1,A000000180332,9.847619
2,A000000177513,9.240000
3,A000000113701,9.725926
4,A000000182399,8.285714
...,...,...
673,A000000169449,9.386047
674,A000000169450,9.541315
675,A000000169445,9.337423
676,A000000169443,9.448276


#### 검증

In [121]:
# 탑리뷰어(1~1000) 상위,중위,하위로 나눠서 리뷰길이차이 (편차) 구하기

#탑리뷰어(1~2000) 상위,중위,하위로 나눈 칼럼 추가
my_goods_df['topRvrRnk_rank'] = pd.cut(my_goods_df['topRvrRnk'], bins=[0, 333, 666, 1000], labels=[1, 2, 3])

#상위 리뷰어와 하위 리뷰어 차이 (리뷰 길이 편차)
high_review_length = my_goods_df.loc[my_goods_df['topRvrRnk'] == 1]['cont_length'].mean()
low_review_length = my_goods_df.loc[my_goods_df['topRvrRnk'] == 3]['cont_length'].mean()

diff_review_length = high_review_length - low_review_length
print(diff_review_length)

# 오히려 하위 탑리뷰어들의 평균 리뷰 길이가 더 길다는 것을 알 수 있다.

-32.9161888111888
